In [1]:
import pandas as pd
import numpy as np
from io import StringIO

#NOTES
#We are losing some position titles in the table format - probably not an issue but will need to grab that from SCTo
#Get rid of info after comma in names
#

In [2]:
def squeeze_nan_left(x):
            original_columns = x.index.tolist()
            squeezed = x.dropna()
            squeezed.index = [original_columns[n] for n in range(squeezed.count())]
            return squeezed.reindex(original_columns, fill_value=np.nan)

In [3]:
def SCT_table_format(SCT):
            #SCT = SCT.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

            for i in range(0,SCT.shape[0]):
                for j in range(0,SCT.shape[1]):
                    if type(SCT.iloc[i,j]) == str:
                        if SCT.iloc[i,j] == "-" or SCT.iloc[i,j] == "—":
                            SCT.iloc[i,j] = 0
                        elif '(' in SCT.iloc[i,j]:
                            tempstr = SCT.iloc[i,j].split("(")
                            tempstr2 = tempstr[0]
                            SCT.iloc[i,j] = tempstr2
                        elif len(SCT.iloc[i,j]) < 2:
                            SCT.iloc[i,j] = np.nan             

            SCT = SCT.replace('', np.nan)
            SCT.iloc[:,0] = SCT.iloc[:,0].fillna(value = '')
            SCT = SCT.fillna(value = '')

            for i in range(0,SCT.shape[0]):
                if SCT.iloc[i, SCT.shape[1]-1] == '':
                    rightslip = ['',SCT.iloc[i]]
                    rightslip = rightslip[:-1]
                    SCT.iloc[i] = rightslip

            SCT.replace('',np.nan,inplace = True)
            SCT = SCT.dropna()

            SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 1)

            return SCT

In [4]:
#Everi - https://www.sec.gov/Archives/edgar/data/1318568/000155837017002779/evri-20170523xdef14a.htm
#MNTA - https://www.sec.gov/Archives/edgar/data/1235010/000123501017000006/mnta2017proxystatement.htm

url = 'https://www.sec.gov/Archives/edgar/data/1410939/000104746917002856/a2231921zdef14a.htm#ds46201_summary_compensation_table'

filing = pd.read_html(url)
#filing[:]
#168 for Everi
#128 for MNTA
#20 for OPHT

sctind = 20
SCTo = filing[sctind]
SCTo

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Name and principal position,NaN,Year,NaN,Salary ($),NaN,Bonus ($)(1),NaN,Stock Awards ($)(2),NaN,...,Total ($),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"David R. Guyer, M.D.(5)",NaN,NaN,2016.0,NaN,NaN,625200,NaN,NaN,—,...,NaN,NaN,325104,NaN,NaN,10405,NaN,NaN,8700989.0,NaN
3,Chief Executive,NaN,NaN,2015.0,NaN,NaN,600000,NaN,NaN,—,...,NaN,NaN,486000,NaN,NaN,18758,NaN,NaN,4866882.0,NaN
4,Officer,NaN,NaN,2014.0,NaN,NaN,545000,NaN,NaN,—,...,NaN,NaN,523200,NaN,NaN,8000,NaN,NaN,5060110.0,NaN
5,"Samir C. Patel, M.D.(6)",NaN,NaN,2016.0,NaN,NaN,591231,NaN,NaN,—,...,NaN,NaN,325104,NaN,NaN,31455,NaN,NaN,8688070.0,NaN
6,Former President,NaN,NaN,2015.0,NaN,NaN,486450,NaN,NaN,—,...,NaN,NaN,361190,NaN,NaN,9038,NaN,NaN,7681275.0,NaN
7,NaN,NaN,NaN,2014.0,NaN,NaN,470000,NaN,NaN,60000,...,NaN,NaN,376000,NaN,NaN,—,NaN,NaN,4093128.0,NaN
8,Glenn P. Sblendorio Sblendorio(7),NaN,NaN,2016.0,NaN,NaN,373154,NaN,NaN,100000,...,NaN,NaN,163350,NaN,NaN,98356,NaN,NaN,8878904.0,NaN
9,President and,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#Table Formatting tests

#initial format, take out nan rows/cols and squeeze nan left
SCT = SCTo.apply(squeeze_nan_left, axis=1).dropna(axis = 0, how = 'all').dropna(axis = 1, how = 'all') 

#secondary format - fill dashes with 0s, remove symbols and parenthesis objects
for i in range(0,SCT.shape[0]):
    for j in range(0,SCT.shape[1]):
        if type(SCT.iloc[i,j]) == str:
            if SCT.iloc[i,j] == "-" or SCT.iloc[i,j] == "—":
                SCT.iloc[i,j] = 0
            elif '(' in SCT.iloc[i,j]:
                tempstr = SCT.iloc[i,j].split("(")
                tempstr2 = tempstr[0]
                SCT.iloc[i,j] = tempstr2
            elif len(SCT.iloc[i,j]) < 2:
                SCT.iloc[i,j] = np.nan             

#clean up again                
SCT = SCT.replace('', np.nan)
SCT.iloc[:,0] = SCT.iloc[:,0].fillna(value = '')
SCT = SCT.fillna(value = '')

#correct for years that are bumped too far left
for i in range(0,SCT.shape[0]):
    if SCT.iloc[i, SCT.shape[1]-1] == '':
        rightslip = ['',SCT.iloc[i]]
        rightslip = rightslip[:-1]
        SCT.iloc[i] = rightslip

#clean up again        
SCT.replace('',np.nan,inplace = True)
SCT = SCT.dropna()

#update column names
SCT = SCT.rename(columns=SCT.iloc[0]).drop(index = 1)                

SCT = SCT.reset_index(drop=True)
SCT

,Name and principal position,Year,Salary,Bonus,Stock Awards,Option Awards,Non- Equity Incentive Plan Compensation,All Other Compensation,Total
0,"David R. Guyer, M.D.",2016,625200,0,3291750,4448530,325104,10405,8.70099e+06
1,Chief Executive,2015,600000,0,1048800,2713323,486000,18758,4.86688e+06
2,Officer,2014,545000,0,0,3983910,523200,8000,5.06011e+06
3,"Samir C. Patel, M.D.",2016,591231,0,3291750,4448530,325104,31455,8.68807e+06
4,Former President,2015,486450,0,4511600,2312997,361190,9038,7.68128e+06
5,Glenn P. Sblendorio Sblendorio,2016,373154,100000,3887705,4256339,163350,98356,8.8789e+06
6,Barbara A. Wood,2016,404620,0,632016,899190,129478,17337,2.08264e+06
7,"Senior Vice President,",2015,390940,0,250800,667211,211110,8000,1.52806e+06
8,General Counsel,2014,372311,60000,0,335815,240000,8000,1.01613e+06
9,Henric B. Bjarke,2016,379380,0,695996,529211,121402,97856,1.82384e+06


In [83]:
#creating NEO name/position lists - done after table format
#will need to grab posisitons from SCTo
#consider making a version that chops off duplicate names, middle initial, endings like M.D.

NEO_Names = []


for i in range(0,SCT.shape[0]):
    if SCT.iloc[i,1] == SCT.iloc[:,1].max():
        NEO_Names.append(SCT.iloc[i,0])
        
        
NEO_Combine = pd.DataFrame(NEO_Names)
NEO_Combine.columns = ['Name']
SCTo_Positions = SCTo.iloc[:,0].dropna()
NEO_Combine['Position'] = ''
SCTo_Positions = SCTo_Positions.reset_index(drop=True)
SCTo_Positions
#NEO_Combine

0                           Name and principal position
1                               David R. Guyer, M.D.(5)
2                                       Chief Executive
3                                               Officer
4                               Samir C. Patel, M.D.(6)
5                                      Former President
6                     Glenn P. Sblendorio Sblendorio(7)
7                                         President and
8     Former Chief Operating Officer, Chief Financia...
9                                       Barbara A. Wood
10                               Senior Vice President,
11                                      General Counsel
12                                  Henric B. Bjarke(8)
13                                   Former Senior Vice
14                                     President, Chief
15                                   Commercial Officer
16                                  Michael G. Atieh(9)
17                                     Former Ex

In [88]:
NEO_Combine['Position'] = ''

k = 0
skip = 0

for i in range(1, len(SCTo_Positions)):
    if SCTo_Positions[i] in NEO_Names != -1:
        k = k + 1
        i = i + 1
    if NEO_Combine.iloc[k,1] == '':
            NEO_Combine.iloc[k,1] = SCTo_Positions[i]
    else:
        NEO_Combine.iloc[k,1] = NEO_Combine.iloc[k,1] + ' ' + SCTo_Positions[i]


NEO_Combine

,Name,Position
0,"David R. Guyer, M.D.","David R. Guyer, M.D.(5) Chief Executive Office..."
1,"Samir C. Patel, M.D.","Senior Vice President, Senior Vice President, ..."
2,Glenn P. Sblendorio Sblendorio,
3,Barbara A. Wood,
4,Henric B. Bjarke,
5,Michael G. Atieh,


In [ ]:
 for j in range(1,len(NEO_Names)):
        if SCTo_Positions[i].find(NEO_Names[j]) != -1:
            k = k + 1
            skip = 1
    else:
        if NEO_Combine.iloc[k,1] == '':
            NEO_Combine.iloc[k,1] = SCTo_Positions[i]
            skip = 0
        else:
            NEO_Combine.iloc[k,1] = NEO_Combine.iloc[k,1] + ' ' + SCTo_Positions[i]
            skip = 0

In [90]:
NEO_Combine.iloc[0,1]

'David R. Guyer, M.D.(5) Chief Executive Officer Samir C. Patel, M.D.(6) Former President Glenn P. Sblendorio Sblendorio(7) President and Former Chief Operating Officer, Chief Financial Officer, Treasurer'